# MODELADO CON XGBOOST

Se tomarán dos casos:

  - Primero: modelar el estado de ánimo
  - Segundo: Modelar el puntaje de estrés

## 2. Modelo predictorio para el puntaje de estrés

In [1]:
!pip install xgboost
!pip install optuna

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import optuna
import pickle
import json

from utils import *
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [6]:
df_estres = pd.read_csv('SWELLdata.csv', decimal=",", delimiter=";")

In [8]:
formateo_fechas(df_estres)

,PP,Blok,Condition,timestamp,Valence_rc,Arousal_rc,Dominance,Stress,MentalEffort,MentalDemand,...,WristLeft_HandLeftPlaneYZAxisZstdv,ShoulderRight_ElbowRightPlaneZXAxisXstdv,ShoulderRight_ElbowRightPlaneXYAxisYstdv,ShoulderRight_ElbowRightPlaneYZAxisZstdv,ElbowRight_WristRightPlaneZXAxisXstdv,ElbowRight_WristRightPlaneXYAxisYstdv,ElbowRight_WristRightPlaneYZAxisZstdv,WristRight_HandRightPlaneZXAxisXstdv,WristRight_HandRightPlaneXYAxisYstdv,WristRight_HandRightKinectZAxisstdv
0,PP1,1,R,2012-09-18 13:16:00,NaN,NaN,NaN,NaN,NaN,NaN,...,166.872086,22.946337,5.605230,13.237048,30.932939,102.460262,8.444481,54.436879,75.886750,155.734343
1,PP1,1,R,2012-09-18 13:17:00,NaN,NaN,NaN,NaN,NaN,NaN,...,58.237714,7.194860,2.468881,3.886001,2.000359,3.431337,1.160298,4.579341,2.868843,171.486911
2,PP1,1,R,2012-09-18 13:18:00,NaN,NaN,NaN,NaN,NaN,NaN,...,64.548824,14.726365,2.953021,9.185623,13.379140,48.162012,30.508877,23.013755,30.629061,149.748809
3,PP1,1,R,2012-09-18 13:19:00,NaN,NaN,NaN,NaN,NaN,NaN,...,38.209895,2.542686,1.564318,2.453674,4.796273,8.223865,1.114906,3.854074,4.048611,173.149430
4,PP1,1,R,2012-09-18 13:20:00,NaN,NaN,NaN,NaN,NaN,NaN,...,80.171422,9.905241,20.342488,5.552234,7.198096,22.942488,46.386278,17.514534,29.702675,165.368301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,PP25,3,T,2012-11-07 16:15:00,9.0,8.0,9.0,0.3,2.7,3.6,...,103.650684,36.355940,5.323538,17.908446,56.076888,124.607529,15.482486,87.838259,158.284130,38.905423
3135,PP25,3,T,2012-11-07 16:16:00,9.0,8.0,9.0,0.3,2.7,3.6,...,26.334438,14.714479,1.848907,3.908359,34.878709,159.984897,5.233569,63.512098,164.607803,27.895148
3136,PP25,3,T,2012-11-07 16:17:00,9.0,8.0,9.0,0.3,2.7,3.6,...,15.970939,22.742133,3.048023,6.468803,53.988308,169.548392,10.757358,60.488127,142.557094,21.761699
3137,PP25,3,T,2012-11-07 16:18:00,9.0,8.0,9.0,0.3,2.7,3.6,...,38.797002,28.595668,27.981837,8.424373,43.183524,55.599939,14.620391,72.590531,163.087296,28.944050


In [9]:
clasificar_momento_dia(df_estres)

,PP,Blok,Condition,timestamp,Valence_rc,Arousal_rc,Dominance,Stress,MentalEffort,MentalDemand,...,ShoulderRight_ElbowRightPlaneZXAxisXstdv,ShoulderRight_ElbowRightPlaneXYAxisYstdv,ShoulderRight_ElbowRightPlaneYZAxisZstdv,ElbowRight_WristRightPlaneZXAxisXstdv,ElbowRight_WristRightPlaneXYAxisYstdv,ElbowRight_WristRightPlaneYZAxisZstdv,WristRight_HandRightPlaneZXAxisXstdv,WristRight_HandRightPlaneXYAxisYstdv,WristRight_HandRightKinectZAxisstdv,momento_dia
0,PP1,1,R,2012-09-18 13:16:00,NaN,NaN,NaN,NaN,NaN,NaN,...,22.946337,5.605230,13.237048,30.932939,102.460262,8.444481,54.436879,75.886750,155.734343,tarde
1,PP1,1,R,2012-09-18 13:17:00,NaN,NaN,NaN,NaN,NaN,NaN,...,7.194860,2.468881,3.886001,2.000359,3.431337,1.160298,4.579341,2.868843,171.486911,tarde
2,PP1,1,R,2012-09-18 13:18:00,NaN,NaN,NaN,NaN,NaN,NaN,...,14.726365,2.953021,9.185623,13.379140,48.162012,30.508877,23.013755,30.629061,149.748809,tarde
3,PP1,1,R,2012-09-18 13:19:00,NaN,NaN,NaN,NaN,NaN,NaN,...,2.542686,1.564318,2.453674,4.796273,8.223865,1.114906,3.854074,4.048611,173.149430,tarde
4,PP1,1,R,2012-09-18 13:20:00,NaN,NaN,NaN,NaN,NaN,NaN,...,9.905241,20.342488,5.552234,7.198096,22.942488,46.386278,17.514534,29.702675,165.368301,tarde
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,PP25,3,T,2012-11-07 16:15:00,9.0,8.0,9.0,0.3,2.7,3.6,...,36.355940,5.323538,17.908446,56.076888,124.607529,15.482486,87.838259,158.284130,38.905423,tarde
3135,PP25,3,T,2012-11-07 16:16:00,9.0,8.0,9.0,0.3,2.7,3.6,...,14.714479,1.848907,3.908359,34.878709,159.984897,5.233569,63.512098,164.607803,27.895148,tarde
3136,PP25,3,T,2012-11-07 16:17:00,9.0,8.0,9.0,0.3,2.7,3.6,...,22.742133,3.048023,6.468803,53.988308,169.548392,10.757358,60.488127,142.557094,21.761699,tarde
3137,PP25,3,T,2012-11-07 16:18:00,9.0,8.0,9.0,0.3,2.7,3.6,...,28.595668,27.981837,8.424373,43.183524,55.599939,14.620391,72.590531,163.087296,28.944050,tarde


In [10]:
columnas_a_eliminar = ['timestamp', 'PP']
df_estres = eliminar_columnas(df_estres, columnas_a_eliminar)

Se eliminaron las columnas: ['timestamp', 'PP']


In [11]:
#Filtramos para condición R no esté
df_estres = df_estres[df_estres['Condition'] != 'R']

In [12]:
#Mover columna momento_día a la cuarta posición
cols = list(df_estres.columns)
momento_dia_index = cols.index('momento_dia')
cols.pop(momento_dia_index)
cols.insert(3, 'momento_dia')
df_estres = df_estres[cols]

In [13]:
#Filtramos para condición momento_día_noche no este (solo hay 2)
df_estres = df_estres[df_estres['momento_dia'] != 'noche']

In [14]:
#Features y varible target
X = df_estres.drop('Stress', axis=1)
y = df_estres['Stress']

In [15]:
#Separar train, eval y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

In [16]:
#Análisis exploratorio X_Train e y_train
pd.set_option("display.max_rows", None)
#Anlanlisis valores nulos
evaluar_datos_faltantes(X_train)

,Columna,Valores Faltantes,Porcentaje (%)
HR,HR,892,51.920838
RMSSD,RMSSD,892,51.920838
ShoulderCenter_ShoulderLeftPlaneZXAxisXstdv,ShoulderCenter_ShoulderLeftPlaneZXAxisXstdv,339,19.732247
ShoulderLeft_ElbowLeftPlaneYZAxisZstdv,ShoulderLeft_ElbowLeftPlaneYZAxisZstdv,329,19.150175
SCL,SCL,326,18.975553
ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv,ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv,305,17.753201
leftShoulderAnglestdv,leftShoulderAnglestdv,305,17.753201
ShoulderCenter_ShoulderLeftPlaneYZAxisZstdv,ShoulderCenter_ShoulderLeftPlaneYZAxisZstdv,273,15.890570
WristLeft_HandLeftPlaneYZAxisZstdv,WristLeft_HandLeftPlaneYZAxisZstdv,264,15.366705
ShoulderLeft_ElbowLeftPlaneXYAxisYstdv,ShoulderLeft_ElbowLeftPlaneXYAxisYstdv,261,15.192084


In [17]:
y_train.isnull().sum()

np.int64(0)

In [18]:
columnas_a_eliminar = ['RMSSD', 'RMSSD']
X_train = eliminar_columnas(X_train, columnas_a_eliminar)

Se eliminaron las columnas: ['RMSSD', 'RMSSD']


In [19]:
#Imputación con media, mediana o moda de nulo en X_train
stats_X_train = X_train.describe().T[['mean', '50%']].assign(mode=X_train.mode().iloc[0])

imputed_values = {}

for col in X_train.columns:
    if col in stats_X_train.index:  # Asegurar que la columna tiene estadísticas disponibles
        moda = stats_X_train.loc[col, 'mode']
        mediana = stats_X_train.loc[col, '50%']
        media = stats_X_train.loc[col, 'mean']

        if moda != 0:
            imputed_values[col] = X_train[col].fillna(moda)
        elif mediana != 0:
            imputed_values[col] = X_train[col].fillna(mediana)
        else:
            imputed_values[col] = X_train[col].fillna(media)

# Aplicar imputaciones en un solo paso sin modificar directamente el DataFrame original
X_train = X_train.assign(**imputed_values)

In [20]:
X_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1718 entries, 2616 to 944
Data columns (total 169 columns):
 #    Column                                                    Dtype  
---   ------                                                    -----  
 0    Blok                                                      int64  
 1    Condition                                                 object 
 2    Valence_rc                                                float64
 3    momento_dia                                               object 
 4    Arousal_rc                                                float64
 5    Dominance                                                 float64
 6    MentalEffort                                              float64
 7    MentalDemand                                              float64
 8    PhysicalDemand                                            float64
 9    TemporalDemand                                            float64
 10   Effort                   

In [21]:
#Convertimos variables categoricas a tipo category
for col in ['Blok', 'Condition', 'momento_dia']:
  X_train[col] = X_train[col].astype('category')

In [22]:
#Encoder variables categoricas. Pocas categorias. One_hot encoding
X_train = pd.get_dummies(X_train, columns=['Blok', 'Condition', 'momento_dia'])

In [23]:
#Punto de control
X_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1718 entries, 2616 to 944
Data columns (total 174 columns):
 #    Column                                                    Dtype  
---   ------                                                    -----  
 0    Valence_rc                                                float64
 1    Arousal_rc                                                float64
 2    Dominance                                                 float64
 3    MentalEffort                                              float64
 4    MentalDemand                                              float64
 5    PhysicalDemand                                            float64
 6    TemporalDemand                                            float64
 7    Effort                                                    float64
 8    Performance_rc                                            float64
 9    Frustration                                               float64
 10   NasaTLX                  

In [24]:
#Correlación entre columnas numéricas
#X_train_num_cols = X_train.select_dtypes(include=['number']).columns
#correlacion(X_train,X_train_num_cols)

In [25]:
#Correlación con la variable objetivo
X_train_with_stress = X_train.copy()
X_train_with_stress['Stress'] = y_train

# Calcular la matriz de correlación solo para las columnas numéricas
X_train_num = X_train_with_stress.select_dtypes(include=['number','bool'])
correlation_matrix = X_train_num.corr()

# Ordenar la correlación respecto a la variable objetivo "Stress"
correlation_with_stress = correlation_matrix["Stress"].sort_values(ascending=False)

# Mostrar las 20 variables más correlacionadas con Stress (o menos si hay menos de 20)
print(correlation_with_stress)

Stress                                                      1.000000
NasaTLX                                                     0.548212
Frustration                                                 0.525831
TemporalDemand                                              0.489234
MentalDemand                                                0.475730
Arousal_rc                                                  0.385589
MentalEffort                                                0.373157
PhysicalDemand                                              0.269737
SAu06_CheekRaiser                                           0.214456
Effort                                                      0.206029
SleftEyebrowLowered                                         0.182530
SAu10_UpperLipRaiser                                        0.173820
SAu43_EyesClosed                                            0.162730
Condition_T                                                 0.154453
SCL                               

In [26]:
#Selección de features según correlación y objetivo
#Si hay muchas variables con correlación muy baja (~0 o <0.05), podemos eliminarlas porque no aportan información.
#Si hay variables con alta correlación entre sí (>0.9), eliminamos una para evitar redundancia.
#Si hay correlación negativa fuerte (<-0.3), podría indicar una relación inversa interesante.


## Identificar columnas con baja correlación con "Stress"
#low_corr_cols = correlation_matrix["Stress"][abs(correlation_matrix["Stress"]) < 0.05].index.tolist()
## Filtrar solo las columnas que existen en ambos datasets (X_train y X_test)
#low_corr_cols = [col for col in low_corr_cols if col in X_train.columns and col in X_test.columns]
## Eliminar las columnas filtradas
#X_train.drop(columns=low_corr_cols, inplace=True)
#print(f"Eliminadas {len(low_corr_cols)} columnas con baja correlación con Price.")




In [27]:
# Obtener la matriz de correlación absoluta solo para las columnas numéricas
X_train_num = X_train.select_dtypes(include=['number'])
corr_matrix = X_train_num.corr().abs()

# Eliminar la columna 'Stress' de la matriz de correlación (si está presente)
if 'Stress' in corr_matrix.columns:
    corr_matrix = corr_matrix.drop('Stress', axis=1).drop('Stress', axis=0)

# Seleccionar las columnas con correlación > 0.9 (excepto la diagonal)
high_corr_cols = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] > 0.9:
            colname = corr_matrix.columns[i]
            high_corr_cols.add(colname)

# Eliminar las columnas redundantes en X_train
X_train.drop(columns=high_corr_cols, inplace=True)

print(f"Eliminadas {len(high_corr_cols)} columnas altamente correlacionadas entre sí (sin considerar 'Stress').")
print(f"Columnas eliminadas: {list(high_corr_cols)}")

Eliminadas 16 columnas altamente correlacionadas entre sí (sin considerar 'Stress').
Columnas eliminadas: ['ShoulderCenter_ShoulderRightPlaneXYAxisYstdv', 'SAu25_LipsPart', 'SnChars', 'HipCenter_SpinePlaneXYAxisYavg', 'SnSpaces', 'ShoulderRight_ElbowRightPlaneXYAxisYavg', 'ElbowLeft_WristLeftWristLeft_HandLeftavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYavg', 'HipCenter_SpinePlaneYZAxisZavg', 'ShoulderCenter_ShoulderRightPlaneYZAxisZavg', 'Spine_ShoulderCenterPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderRightPlaneXYAxisYavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv', 'Spine_ShoulderCenterPlaneYZAxisZavg', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg']


In [28]:
print(f"Columnas eliminadas: {list(high_corr_cols)}")

Columnas eliminadas: ['ShoulderCenter_ShoulderRightPlaneXYAxisYstdv', 'SAu25_LipsPart', 'SnChars', 'HipCenter_SpinePlaneXYAxisYavg', 'SnSpaces', 'ShoulderRight_ElbowRightPlaneXYAxisYavg', 'ElbowLeft_WristLeftWristLeft_HandLeftavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYavg', 'HipCenter_SpinePlaneYZAxisZavg', 'ShoulderCenter_ShoulderRightPlaneYZAxisZavg', 'Spine_ShoulderCenterPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderRightPlaneXYAxisYavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv', 'Spine_ShoulderCenterPlaneYZAxisZavg', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg']


In [29]:
#Análisis exploratorio X_eval e y_eval
pd.set_option("display.max_rows", None)
#Anlanlisis valores nulos
evaluar_datos_faltantes(X_eval)

,Columna,Valores Faltantes,Porcentaje (%)
HR,HR,245,56.976744
RMSSD,RMSSD,245,56.976744
SCL,SCL,89,20.697674
ShoulderCenter_ShoulderLeftPlaneZXAxisXstdv,ShoulderCenter_ShoulderLeftPlaneZXAxisXstdv,89,20.697674
ShoulderLeft_ElbowLeftPlaneYZAxisZstdv,ShoulderLeft_ElbowLeftPlaneYZAxisZstdv,77,17.906977
ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv,ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv,67,15.581395
leftShoulderAnglestdv,leftShoulderAnglestdv,67,15.581395
ShoulderCenter_ShoulderLeftPlaneYZAxisZstdv,ShoulderCenter_ShoulderLeftPlaneYZAxisZstdv,61,14.186047
ShoulderLeft_ElbowLeftPlaneXYAxisYstdv,ShoulderLeft_ElbowLeftPlaneXYAxisYstdv,60,13.953488
ElbowLeft_WristLeftPlaneYZAxisZstdv,ElbowLeft_WristLeftPlaneYZAxisZstdv,59,13.720930


In [30]:
y_eval.isnull().sum()

np.int64(0)

In [31]:
#Eliminamos las mismas columnas que en X_eval
columnas_a_eliminar = ['RMSSD', 'RMSSD','HipCenter_SpinePlaneYZAxisZavg', 'Spine_ShoulderCenterPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftavg', 'Spine_ShoulderCenterPlaneYZAxisZavg', 'SnChars', 'HipCenter_SpinePlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneYZAxisZavg', 'SAu25_LipsPart', 'ShoulderRight_ElbowRightPlaneXYAxisYavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg', 'ShoulderCenter_ShoulderRightPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderRightPlaneXYAxisYavg', 'SnSpaces', 'ElbowLeft_WristLeftWristLeft_HandLeftavg']
X_eval = eliminar_columnas(X_eval, columnas_a_eliminar)

Se eliminaron las columnas: ['RMSSD', 'RMSSD', 'HipCenter_SpinePlaneYZAxisZavg', 'Spine_ShoulderCenterPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftavg', 'Spine_ShoulderCenterPlaneYZAxisZavg', 'SnChars', 'HipCenter_SpinePlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneYZAxisZavg', 'SAu25_LipsPart', 'ShoulderRight_ElbowRightPlaneXYAxisYavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg', 'ShoulderCenter_ShoulderRightPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderRightPlaneXYAxisYavg', 'SnSpaces', 'ElbowLeft_WristLeftWristLeft_HandLeftavg']


In [32]:
#Imputación con media, mediana o moda de nulo en X_eval
stats_X_eval = X_eval.describe().T[['mean', '50%']].assign(mode=X_eval.mode().iloc[0])

imputed_values = {}

for col in X_eval.columns:
    if col in stats_X_eval.index:  # Asegurar que la columna tiene estadísticas disponibles
        moda = stats_X_eval.loc[col, 'mode']
        mediana = stats_X_eval.loc[col, '50%']
        media = stats_X_eval.loc[col, 'mean']

        if moda != 0:
            imputed_values[col] = X_eval[col].fillna(moda)
        elif mediana != 0:
            imputed_values[col] = X_eval[col].fillna(mediana)
        else:
            imputed_values[col] = X_eval[col].fillna(media)

# Aplicar imputaciones en un solo paso sin modificar directamente el DataFrame original
X_eval = X_eval.assign(**imputed_values)

In [33]:
for col in ['Blok', 'Condition', 'momento_dia']:
  X_eval[col] = X_eval[col].astype('category')

In [34]:
#Encoder variables categoricas. Pocas categorias. One_hot encoding
X_eval = pd.get_dummies(X_eval, columns=['Blok', 'Condition', 'momento_dia'])

In [35]:
#Comparar si tienen las mismas columnas en X_train y X_eval
# Obtener las columnas de cada DataFrame
X_eval_cols = X_eval.columns.tolist()
X_train_cols = X_train.columns.tolist()

# Encontrar las columnas que están en ambos DataFrames
common_cols = list(set(X_eval_cols) & set(X_train_cols))

# Encontrar las columnas que están en X_eval pero no en X_train
eval_only_cols = list(set(X_eval_cols) - set(X_train_cols))

# Encontrar las columnas que están en X_train pero no en X_eval
train_only_cols = list(set(X_train_cols) - set(X_eval_cols))

# Imprimir los resultados
print("Columnas comunes:", common_cols)
print("Columnas solo en X_eval:", eval_only_cols)
print("Columnas solo en X_train:", train_only_cols)

Columnas comunes: ['Spine_ShoulderCenterShoulderCenter_Headavg', 'ElbowRight_WristRightPlaneYZAxisZavg', 'SnMouseAct', 'ShoulderLeft_ElbowLeftElbowLeft_WristLeftavg', 'ElbowRight_WristRightWristRight_HandRightavg', 'Spine_ShoulderCenterShoulderCenter_Headstdv', 'HipCenter_SpinePlaneYZAxisZstdv', 'Dominance', 'SAu10_UpperLipRaiser', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftstdv', 'MentalDemand', 'SnTabfocusChange', 'HipCenter_SpinePlaneZXAxisXavg', 'SAu43_EyesClosed', 'Sscared', 'Effort', 'SAu17_ChinRaiser', 'Blok_2', 'WristRight_HandRightKinectZAxisstdv', 'avgDepthstdv', 'SnDirectionKeys', 'leanAngleavg', 'WristLeft_HandLeftPlaneXYAxisYstdv', 'SnWheel', 'SnSpecialKeys', 'leanAnglestdv', 'Spine_ShoulderCenterPlaneZXAxisXavg', 'WristLeft_HandLeftPlaneYZAxisZavg', 'WristLeft_HandLeftPlaneZXAxisXavg', 'Sangry', 'SyHeadOrientation', 'Svalence', 'rightShoulderAngleavg', 'SAu06_CheekRaiser', 'ShoulderCenter_ShoulderRightPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderLeftPlaneZXAxisXavg',

In [ ]:
#Optuna (Prueba_1)
#def objective_xgb_stress(trial):
#    params = {
#        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
#        "max_depth": trial.suggest_int("max_depth", 3, 10),
#        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
#        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
#        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
#        # Regularización L2
#        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
#        # Parámetros de complejidad
#        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
#        "gamma": trial.suggest_float("gamma", 0.0, 1.0)
#    }
#
#    model = xgb.XGBRegressor(objective="reg:squarederror", **params)
#    # Early stopping con cross-validation
#    score = cross_val_score(
#                model,
#                X_train,
#                y_train,
#                cv=3,
#                scoring="neg_mean_absolute_error",
#                fit_params={"early_stopping_rounds": 10, "eval_set": [(X_eval, y_eval)]},
#            ).mean()
#    return -score
#
#study_xgb_stress = optuna.create_study(direction="minimize")
#study_xgb_stress.optimize(objective_xgb_stress, n_trials=20)
#best_params_xgb_stress = study_xgb_stress.best_params

In [36]:
#Optuna Prueba_2
import optuna
import xgboost as xgb
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def objective_xgb_stress(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 1.0),
    }

    model = xgb.XGBRegressor(objective="reg:squarederror", **params)

    # Validación cruzada
    kf = KFold(n_splits=3)

    best_score = float('inf')  # Inicializar con infinito para la primera comparación
    n_rounds_without_improvement = 0

    for i, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], verbose=False)
        score = mean_absolute_error(y_val_fold, model.predict(X_val_fold))

        if score < best_score:  # Si el score mejora
            best_score = score
            n_rounds_without_improvement = 0
        else:
            n_rounds_without_improvement += 1

        # Early stopping con Optuna
        if n_rounds_without_improvement >= 10:  # Detener si no hay mejora en 10 rondas
            trial.report(best_score, i)  # Reportar el mejor score alcanzado

            if trial.should_prune():  # Verificar si Optuna recomienda detener el trial
                raise optuna.TrialPruned()
            break

    return best_score  # Devolver el mejor score alcanzado

# Crear el estudio de Optuna y optimizar
study_xgb_stress = optuna.create_study(
    direction="minimize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10, interval_steps=1) # Agregar un pruner (opcional)
)
study_xgb_stress.optimize(objective_xgb_stress, n_trials=20)

# Obtener los mejores hiperparámetros
best_params_xgb_stress = study_xgb_stress.best_params

[I 2025-03-24 19:42:04,166] A new study created in memory with name: no-name-84e41dae-f83c-4b67-a5d3-ed9729832861
[I 2025-03-24 19:42:06,412] Trial 0 finished with value: 0.09830932291803629 and parameters: {'n_estimators': 214, 'max_depth': 10, 'learning_rate': 0.29460248482926255, 'subsample': 0.6159868656583511, 'colsample_bytree': 0.6986273859646965, 'reg_lambda': 4.03515061372749e-05, 'min_child_weight': 6, 'gamma': 0.1494538361899076}. Best is trial 0 with value: 0.09830932291803629.
[I 2025-03-24 19:42:07,715] Trial 1 finished with value: 0.06791042606061877 and parameters: {'n_estimators': 109, 'max_depth': 8, 'learning_rate': 0.279280956909901, 'subsample': 0.8503769147910503, 'colsample_bytree': 0.8080334912475626, 'reg_lambda': 3.060252953153029e-06, 'min_child_weight': 7, 'gamma': 0.37203881573639996}. Best is trial 1 with value: 0.06791042606061877.
[I 2025-03-24 19:42:10,088] Trial 2 finished with value: 0.09888547556198497 and parameters: {'n_estimators': 296, 'max_depth

Prueba_1: [I 2025-03-19 09:12:01,333] Trial 13 finished with value: 0.031598982845048904 and parameters: {'n_estimators': 396, 'max_depth': 10, 'learning_rate': 0.10038571532701207, 'subsample': 0.8821131233679669, 'colsample_bytree': 0.9353893298353234}. Best is trial 13 with value: 0.031598982845048904

In [37]:
best_params_xgb_stress

{'n_estimators': 357,
 'max_depth': 9,
 'learning_rate': 0.2505717003798547,
 'subsample': 0.6673254613786576,
 'colsample_bytree': 0.9952500641622928,
 'reg_lambda': 0.07864597044130107,
 'min_child_weight': 5,
 'gamma': 0.0006624950121453255}

In [38]:
#Entrenar Modelo con los Mejores Parámetros
xgb_stress = xgb.XGBRegressor(objective="reg:squarederror", **best_params_xgb_stress)
xgb_stress.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9952500641622928, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=0.0006624950121453255, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.2505717003798547, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=357, n_jobs=None,
             num_parallel_tree=None, ...)

In [55]:
# Hacer predicciones en el conjunto de entrenamiento
y_pred_train = xgb_stress.predict(X_train)

# Calcular las métricas
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_pred_train)
mae_train = mean_absolute_error(y_train, y_pred_train)

# Imprimir las métricas
print(f"Métricas en el conjunto de entrenamiento:")
print(f"Error Cuadrático Medio (MSE): {mse_train}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse_train}")
print(f"R-cuadrado (R^2): {r2_train}")
print(f"Error Absoluto Medio (MAE): {mae_train}")

Métricas en el conjunto de entrenamiento:
Error Cuadrático Medio (MSE): 5.7871111303533544e-05
Raíz del Error Cuadrático Medio (RMSE): 0.007607306442068279
R-cuadrado (R^2): 0.9999869403260923
Error Absoluto Medio (MAE): 0.005594063557523356


In [39]:
# Hacer predicciones en el conjunto de evaluación
y_pred = xgb_stress.predict(X_eval)

In [40]:
# Calcular las métricas de evaluación
mse_eval = mean_squared_error(y_eval, y_pred)
rmse_eval = np.sqrt(mse_eval)
r2_eval = r2_score(y_eval, y_pred)
mae_eval = mean_absolute_error(y_eval, y_pred)
# Imprimir las métricas
print(f"Error Cuadrático Medio (MSE): {mse_eval}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse_eval}")
print(f"R-cuadrado (R^2): {r2_eval}")
print(f"Error Absoluto Medio (MAE): {mae_eval}")

Error Cuadrático Medio (MSE): 0.020651960976375068
Raíz del Error Cuadrático Medio (RMSE): 0.1437079015794715
R-cuadrado (R^2): 0.9953202874542634
Error Absoluto Medio (MAE): 0.047350412729591514


In [42]:
results_df = pd.DataFrame({'y_eval': y_eval, 'y_pred': y_pred})
print(results_df)

      y_eval    y_pred
2680     1.5  1.566331
1321     6.8  6.745089
641      3.2  3.168046
2269     0.0  0.025217
2767     5.0  5.030398
110      3.5  3.529080
1655     2.0  1.929114
3010     1.3  1.379566
404      3.9  3.897645
606      2.2  2.163290
1897     2.0  2.070928
2040     6.9  6.585558
671      3.2  3.096117
750      3.1  2.978340
2102     5.1  4.987798
320      0.9  1.414219
2089     2.3  2.169241
684      1.3  1.347941
1140     2.8  2.800983
1199     1.8  1.809141
897      2.6  2.663746
2739     4.0  4.006232
238      1.5  1.699627
63       5.9  5.848737
1269     1.4  1.400294
3009     1.3  1.487503
1961     6.0  5.995220
2744     4.0  4.038691
1929     2.0  2.044581
1446     7.4  7.355335
2406     2.2  2.212552
1212     1.8  1.898538
421      3.9  3.833267
2876     5.3  5.310461
1777     1.3  1.231588
1192     1.8  1.792704
1097     3.3  3.283197
2735     4.0  4.026234
969      4.7  4.680027
1858     0.4  0.396632
2209     4.6  4.581157
2249     4.3  4.010626
3123     0.

In [41]:
#Eliminamos las mismas columnas que en X_test
columnas_a_eliminar = ['RMSSD', 'RMSSD','HipCenter_SpinePlaneYZAxisZavg', 'Spine_ShoulderCenterPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftavg', 'Spine_ShoulderCenterPlaneYZAxisZavg', 'SnChars', 'HipCenter_SpinePlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneYZAxisZavg', 'SAu25_LipsPart', 'ShoulderRight_ElbowRightPlaneXYAxisYavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg', 'ShoulderCenter_ShoulderRightPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderRightPlaneXYAxisYavg', 'SnSpaces', 'ElbowLeft_WristLeftWristLeft_HandLeftavg']
X_test = eliminar_columnas(X_test, columnas_a_eliminar)

Se eliminaron las columnas: ['RMSSD', 'RMSSD', 'HipCenter_SpinePlaneYZAxisZavg', 'Spine_ShoulderCenterPlaneYZAxisZstdv', 'ShoulderCenter_ShoulderLeftShoulderLeft_ElbowLeftavg', 'Spine_ShoulderCenterPlaneYZAxisZavg', 'SnChars', 'HipCenter_SpinePlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneYZAxisZavg', 'SAu25_LipsPart', 'ShoulderRight_ElbowRightPlaneXYAxisYavg', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderLeftPlaneXYAxisYavg', 'ShoulderCenter_ShoulderRightPlaneZXAxisXavg', 'ShoulderCenter_ShoulderRightPlaneXYAxisYstdv', 'ShoulderCenter_ShoulderRightPlaneXYAxisYavg', 'SnSpaces', 'ElbowLeft_WristLeftWristLeft_HandLeftavg']


In [42]:
#Imputación con media, mediana o moda de nulo en X_test
stats_X_test = X_test.describe().T[['mean', '50%']].assign(mode=X_test.mode().iloc[0])

imputed_values = {}

for col in X_test.columns:
    if col in stats_X_test.index:  # Asegurar que la columna tiene estadísticas disponibles
        moda = stats_X_test.loc[col, 'mode']
        mediana = stats_X_test.loc[col, '50%']
        media = stats_X_test.loc[col, 'mean']

        if moda != 0:
            imputed_values[col] = X_test[col].fillna(moda)
        elif mediana != 0:
            imputed_values[col] = X_test[col].fillna(mediana)
        else:
            imputed_values[col] = X_test[col].fillna(media)

# Aplicar imputaciones en un solo paso sin modificar directamente el DataFrame original
X_test = X_test.assign(**imputed_values)

In [43]:
for col in ['Blok', 'Condition', 'momento_dia']:
  X_test[col] = X_test[col].astype('category')

In [44]:
X_test = pd.get_dummies(X_test, columns=['Blok', 'Condition', 'momento_dia'])

In [45]:
# Hacer predicciones en el conjunto de prueba
y_pred_test = xgb_stress.predict(X_test)

# Calcular las métricas de evaluación para el conjunto de prueba
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

# Imprimir las métricas para el conjunto de prueba
print(f"Métricas en el conjunto de prueba:")
print(f"Error Cuadrático Medio (MSE): {mse_test}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse_test}")
print(f"R-cuadrado (R^2): {r2_test}")
print(f"Error Absoluto Medio (MAE): {mae_test}")

Métricas en el conjunto de prueba:
Error Cuadrático Medio (MSE): 0.012847409297971901
Raíz del Error Cuadrático Medio (RMSE): 0.11334641281475079
R-cuadrado (R^2): 0.9976039196578549
Error Absoluto Medio (MAE): 0.04545571086998033


In [46]:
# Obtener la importancia de las features
importance = xgb_stress.feature_importances_

# Crear un DataFrame con las features y su importancia
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importance})

# Ordenar el DataFrame por importancia descendente
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

feature_importance

,Feature,Importance
10,NasaTLX,0.193766
6,TemporalDemand,0.152343
9,Frustration,0.137883
3,MentalEffort,0.067620
69,avgDepthavg,0.053632
98,ShoulderRight_ElbowRightPlaneZXAxisXavg,0.048459
4,MentalDemand,0.041433
8,Performance_rc,0.039740
5,PhysicalDemand,0.037251
151,Blok_2,0.036377


In [56]:

# Guardar el modelo en un archivo .pkl
with open("xgb_stress_1.pkl", "wb") as f:
    pickle.dump(xgb_stress, f)

# Guardar los parametros y metricas en un archivo .json
import json

results = {
    "best_params": best_params_xgb_stress,
    "mse_train": mse_train,
    "rmse_train": rmse_train,
    "r2_train": r2_train,
    "mae_train": mae_train,
    "mse_eval": mse_eval,
    "rmse_eval": rmse_eval,
    "r2_eval": r2_eval,
    "mae_eval": mae_eval,
    "mse_test": mse_test,
    "rmse_test": rmse_test,
    "r2_test": r2_test,
    "mae_test": mae_test
}

with open("metricas_y_best_params_xgb_stress_1.json", "w") as f:
    json.dump(results, f)



In [49]:
# 1. Crear DataFrame con la importancia de las features
importance = xgb_stress.feature_importances_
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': importance})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# 2. Crear DataFrame con las métricas
metrics = {
    'Conjunto': ['Train', 'Validation', 'Test'],
    'MSE': [mse_train, mse_eval, mse_test],
    'RMSE': [rmse_train, rmse_eval, rmse_test],
    'R^2': [r2_train, r2_eval, r2_test],
    'MAE': [mae_train, mae_eval, mae_test]
}
metrics_df = pd.DataFrame(metrics)

# 3. Crear DataFrame con los mejores hiperparámetros
best_params_df = pd.DataFrame([study_xgb_stress.best_params], index=['Mejores Hiperparámetros'])

# 4. Concatenar los DataFrames
all_results_df = pd.concat([feature_importance, metrics_df, best_params_df], axis=0) #axis=0 para concatenar verticalmente

# 5. Guardar el DataFrame como un archivo Excel
all_results_df.to_excel('P2_XGB_1.xlsx')